In [21]:
!pip install --upgrade transformers

In [22]:
import nltk # Analyse des données textuelles, comme le tokenization, le stemming, la gestion des stopwords, ...
import random # Génère des nombres aléatoires ce qui permet de mélanger une liste ou sélectionner un élément au hasard
import json # Travailler avec les fichiers JSON
import torch # construire des réseaux de neurones, et entraîner des modèles (DL)
from transformers import BertTokenizer, BertForSequenceClassification # Utiliser des modèles pré-entraînés comme BERT pour divers taches NLP
# Instead of importing from transformers.optimization, import from torch.optim
from torch.optim import AdamW # Utiliser des modèles pré-entraînés comme BERT pour divers taches NLP
from sklearn.preprocessing import LabelEncoder # Ajuster les poids d'un réseau de neurones
from torch.utils.data import DataLoader, TensorDataset  # Convertit le text en valeurs numériques

In [23]:
# Télécharger les ressources nécessaires
nltk.download('punkt')  # diviser du texte en phrases ou en mots
nltk.download('wordnet')  # lemmatisation

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [24]:
# Chargement des intentions
with open('intents.json') as f:
    intents = json.load(f)

In [25]:
# Initialisation du tokenizer BERT
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [26]:
# Préparation des données d'entraînement
patterns = []  # Cette liste va contenir les phrases ou modèles de texte (inputs)
labels = []   # Cette liste va contenir les tags

# Extraction des phrases et des labels
for intent in intents['intents']:   # Accède à la clé intents dans JSON
    for pattern in intent['patterns']:
        patterns.append(pattern)   # Ajoute chaque phrase à la liste patterns
        labels.append(intent['tag'])  #  ajoute l'étiquete tag à la liste labels

In [27]:
# Encodage des labels (tag des intentions)
label_encoder = LabelEncoder()  # transformer des données catégorielles en chiffres uniques
labels = label_encoder.fit_transform(labels)

In [28]:
# Tokenisation des phrases avec le tokenizer BERT
inputs = tokenizer(patterns, padding=True, truncation=True, return_tensors='pt', max_length=50)

# convertir les données textuelles en données exploitable par BERT

In [29]:
# Convertir les labels en tensors
labels = torch.tensor(labels)

In [30]:
# Créer un DataLoader pour l'entraînement du modèle de DL
dataset = TensorDataset(inputs['input_ids'], inputs['attention_mask'], labels)
train_dataloader = DataLoader(dataset, batch_size=8, shuffle=True)

In [31]:
# Charger le modèle pré-entraîné BERT pour la classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(label_encoder.classes_))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
# Utilisation de l'optimizer AdamW pour ajuster les poids d'un modèle au cours de l'entraînement
optimizer = AdamW(model.parameters(), lr=1e-5)

In [33]:
# Entraînement du modèle
model.train()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [34]:
# Boucle d'entraînement

# minimiser la perte sur les données d'entraînement en ajustant ses poids à chaque itération

epochs = 3
for epoch in range(epochs):
    for batch in train_dataloader:
        b_input_ids, b_attention_mask, b_labels = batch
        optimizer.zero_grad()
        outputs = model(b_input_ids, attention_mask=b_attention_mask, labels=b_labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs} - Loss: {loss.item()}")

Epoch 1/3 - Loss: 3.8610947132110596
Epoch 2/3 - Loss: 3.654578685760498
Epoch 3/3 - Loss: 3.779175281524658


# Nouvelle section

In [35]:
# Sauvegarder le modèle entraîné pour une utilisation ulterieure
model.save_pretrained("chatbot_bert_model")
tokenizer.save_pretrained("chatbot_bert_model")

('chatbot_bert_model/tokenizer_config.json',
 'chatbot_bert_model/special_tokens_map.json',
 'chatbot_bert_model/vocab.txt',
 'chatbot_bert_model/added_tokens.json')

# section nvx

In [36]:
# définit une fonction appelée get_response qui cherche une réponse associée
# à une intention donnée (intent) à partir d'un ensemble d'intentions JSON (intents_json)
def get_response(intent, intents_json):
    for item in intents['intents']:
      if item['tag'] == intent:
        return(item["responses"])
    return "désoler, je n'est pas trouver !"

In [37]:
# Fonction principale pour Gradio qui traite les messages de l'utilisateur et génère des
# réponses basées sur les intentions détectées

def chatbot_interface(message,intents):
    predicted_intent = predict_class(message)
    if predicted_intent == "Classe inconnue":
        return "Désolé, je n'ai pas compris votre demande."
    else:
      response = get_response(predicted_intent, intents)
      return response

In [38]:
import torch.nn.functional as F

def predict_class(message):
    inputs = tokenizer(message, return_tensors='pt', padding=True, truncation=True, max_length=50)
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    with torch.no_grad():
        outputs = model(input_ids, attention_mask=attention_mask)

    logits = outputs.logits

    # Calculer les probabilités
    probabilities = F.softmax(logits, dim=1)

    # Obtenir la classe prédite et sa probabilité
    predicted_label = torch.argmax(probabilities, dim=1).item()
    predicted_probability = probabilities[0, predicted_label].item()

    # Définir un seuil
    threshold = 0.6

    # Vérifier si la probabilité dépasse le seuil
    if predicted_probability < threshold:
        return "Classe inconnue"

    # Récupérer le tag de l'intention
    intent = label_encoder.inverse_transform([predicted_label])[0]
    return intent


##### Cette fonction predict_class prend un message utilisateur en entrée, passe ce message dans un modèle d'apprentissage profond (comme BERT), et prédit l'intention correspondante si elle dépasse un certain seuil de confiance. Si la probabilité associée à l'intention prédite est trop basse, elle retourne "Classe inconnue"

In [39]:
!pip install gradio

In [40]:
# configuration d'une interface utilisateur simple avec Gradio pour interagir avec un chatbot basé sur BERT

import gradio as gr
# Créer l'interface Gradio
iface = gr.Interface(
    fn=chatbot_interface,
    inputs="text",
    outputs="text",
    title="Chatbot FIRST AID",
    description="Entrez un message et obtenez une réponse du chatbot basé sur BERT.",
)

/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1017: UserWarning: Expected 2 arguments for function <function chatbot_interface at 0x7aaeb3db39c0>, received 1.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/gradio/utils.py:1021: UserWarning: Expected at least 2 arguments for function <function chatbot_interface at 0x7aaeb3db39c0>, received 1.
  warnings.warn(


In [41]:
iface.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8c1ffd8cb89f37b221.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
